In [23]:
from os import getenv
import sys
import psycopg2
from dotenv import load_dotenv
load_dotenv()

ELEPHANTSQL_DATABASE = getenv('ELEPHANTSQL_DATABASE')
ELEPHANTSQL_USERNAME = getenv('ELEPHANTSQL_USERNAME')
ELEPHANTSQL_PASSWORD = getenv('ELEPHANTSQL_PASSWORD')
ELEPHANTSQL_HOST = getenv('ELEPHANTSQL_HOST')

In [24]:
def connect(DATABASE, USERNAME, PASSWORD, HOST):
    """ Connect to the PostgreSQL database server """
    elephantsql_client = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')

        # Connect to ElephantSQL-hosted PostgreSQL
        elephantsql_client = psycopg2.connect(dbname=DATABASE, user=USERNAME, password=PASSWORD, host=HOST)

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    return elephantsql_client



# Building Inital Table
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

Connecting to the PostgreSQL database...


In [ ]:
def create_table(elephantsql_client):
    ''' Creating a table based on whats being stored on twitoff app'''

    # For the purposes of this I will handwrite the sql commands
    command = '''
        CREATE TABLE IF NOT EXISTS strain_table (id                      SERIAL PRIMARY KEY,
                                                 strain_name             varchar(50),
                                                 strain_link             varchar(100))
                                                
        '''

    command = '''
    CREATE TABLE IF NOT EXISTS user_table (id                   SERIAL PRIMARY KEY,
                                           name                 varchar(30),
                                           email                varchar(30),
                                           password             varchar(30),
                                           saved_strains        varchar(500))
                                            
    '''

    command = '''
    CREATE TABLE IF NOT EXISTS {}_tweets_table (id                 SERIAL PRIMARY KEY,
                                                tweet_text             varchar(500))
                                            
    '''
    
    try:

        # A "cursor", a structure to iterate over db records to perform queries
        cur = elephantsql_client.cursor()

        # Execute commands in order
        cur.execute(command)

        # Close communication with the PostgreSQL database server
        cur.close()

        # Commit the changes
        elephantsql_client.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

def single_insert(elephantsql_client, insert_req):
    """ Execute a single INSERT request """
    cur = elephantsql_client.cursor()
    try:
        cur.execute(insert_req)
        elephantsql_client.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        elephantsql_client.rollback()
        cur.close()
        return 1
    cur.close()